In [ ]:
import pickle

In [ ]:
X,y_hr,y_participant,y_activity = pickle.load(open('../data/heart_rate_tabular_data_ppg_dalia.p','rb'))

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split,LeaveOneGroupOut
from sklearn.metrics import accuracy_score
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import backend as K
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping
from sklearn.model_selection import train_test_split
# import numpy as np
# %matplotlib notebook
from tensorflow.keras.layers import Conv1D,BatchNormalization,Dropout,Input,MaxPooling1D,Flatten,Dense,Input
from tensorflow.keras.models import Model, Sequential

In [ ]:
def get_model(input_shape=(256,4),act='tanh',loss="mae",opt='adam',n_classes=1):
    model =  Sequential()
    model.add(Conv1D(100,10,input_shape=input_shape,activation='linear',kernel_initializer='normal',padding='same'))
    model.add(Conv1D(100,10,input_shape=input_shape,activation='relu',kernel_initializer='normal',padding='same'))
    model.add(MaxPooling1D(2))
    model.add(BatchNormalization())
    model.add(Dropout(.2))
    model.add(Conv1D(100,10,activation='relu',kernel_initializer='normal',padding='same'))
    model.add(MaxPooling1D(2))
    model.add(BatchNormalization())
    model.add(Dropout(.2))
    model.add(Conv1D(100,10,activation='relu',kernel_initializer='normal',padding='same'))
    model.add(MaxPooling1D(2))
    model.add(BatchNormalization())
    model.add(Conv1D(200,10,activation='relu',kernel_initializer='normal',padding='same'))
    model.add(MaxPooling1D(2))
    model.add(BatchNormalization())
    model.add(Dropout(.2))
    model.add(Conv1D(50,2,activation='relu',kernel_initializer='normal',padding='same'))
    model.add(MaxPooling1D(4))
    model.add(BatchNormalization())
    model.add(Dropout(.2))
    model.add(Flatten())
    model.add(Dense(n_classes*20,activation='relu',kernel_initializer='normal'))
#     model.add(tf.keras.layers.Lambda(lambda x: tf.math.l2_normalize(x, axis=1),name='embedding'))
    model.add(Dense(n_classes,activation='relu',kernel_initializer='normal'))
    model.compile(loss=loss,optimizer=opt)
    return model
model =  get_model()
model.summary()

In [ ]:
from keras.models import load_model
logo = LeaveOneGroupOut()
y_final = []
y_final_pred = []
y_activity_final = []
y_participant_final = []

for train_index, test_index in logo.split(X, y_hr, y_participant.reshape(-1)):
    train_x, test_x = X[train_index], X[test_index]
    train_y, test_y = y_hr[train_index], y_hr[test_index]
    train_participant, test_participant = y_participant[train_index], y_participant[test_index]
    train_activity, test_activity = y_activity[train_index], y_activity[test_index]
    train_x, val_x, train_y, val_y = train_test_split(train_x,train_y,test_size = 0.1)
    
    print((train_x.shape, train_y.shape), 
          (val_x.shape, val_y.shape,),
          (test_x.shape, test_y.shape, test_activity.shape, test_participant.shape))    
    
    
    model = get_model()
    filepath = '../model_files/temp_heart_rate_estimator.hdf5'
    checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
    es = EarlyStopping(monitor='val_loss', mode='min', verbose=0,patience=20)
    callbacks_list = [es,checkpoint]
    history = model.fit(train_x,train_y,validation_data=(val_x,val_y), epochs=100, batch_size=500,verbose=1,
          callbacks=callbacks_list,shuffle=True)
    
    
    model = load_model(filepath)
    test_y_pred = model.predict(test_x)
    
    y_final.extend(list(test_y))
    y_final_pred.extend(list(test_y_pred))
    y_activity_final.extend(list(test_activity))
    y_participant_final.extend(list(test_participant))
    
    print(test_participant[0],'Done')
    break
    

In [ ]:
pickle.dump([y_final,y_final_pred,y_activity_final,y_participant_final],
            open('../data/heart_rate_prediction_ppg_dalia.p','wb'))

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
y_final_pred = [i[0] for i in y_final_pred]

In [ ]:
plt.plot(y_final)
plt.plot(y_final_pred)
from sklearn.metrics import mean_absolute_error
mean_absolute_error(y_final,y_final_pred)

In [ ]:
from sklearn.metrics import mean_absolute_error
mean_absolute_error(y_final,y_final_pred)

In [ ]:
import pandas as pd
df = pd.DataFrame({'original':y_final,'predicted':y_final_pred,'activity':y_activity_final})

In [ ]:
df['error'] = df.apply(lambda a:abs(a['original']-a['predicted']),axis=1)

In [ ]:
plt.figure(figsize=(20,10))
sns.boxplot(x='activity',y='error',data=df)
plt.show()

In [ ]:
df.groupby('activity').mean()